In [1]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres
import numpy as np

from hpbandster.optimizers import BOHB as BOHB
import pickle

In [2]:
import sys
sys.path.insert(0, '..')

%run plate_EQS.ipynb


---------------------- SCIANN 0.6.4.5 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [3]:
import Pinn_Worker

In [4]:
valData = [['u.txt'],'ofen']
test_gridObj = SEQ.Grid(3,{'x':0,'y':1,'t':2},[[[-2,2],[-2,2],[0,10]]],10)
configspecs = {
    'denspt':[3,8],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator': ['elu','relu','selu','swish',
                 'tanh','Addons>gelu','Addons>mish',
                          'Addons>softshrink'], 
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000,25000],
    'initial_lr':[1e-4,1e-2]
}




In [5]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.
NS = hpns.NameServer(run_id='example1', host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.
numWorkers = 1
experiment_name = 'plate_exp'
run_id='plate_Gridsearch_1'


workers=[]
for i in range(1,numWorkers+1):
    print('Creating worker')
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=True,
    experiment_name = experiment_name,
    nameserver='127.0.0.1',
    run_id=run_id,
    id=i)
    
    print('Worker created, starting it')
    w.run(background=True)
    print('Worker running, adding it to the list')
    workers.append(w)
    print('Worker added')

Creating worker
Creating client
New experiment. Creating instance.
Parsing validation data
Applying validation data to test grid
Saving PDE and config
Worker ready
Worker created, starting it
Worker running, adding it to the list
Worker added


17:06:11 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x1dd3c560148; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
17:06:11 WORKER: No dispatcher found. Waiting for one to initiate contact.
17:06:11 WORKER: start listening for jobs


In [6]:
bohb = BOHB(  configspace = w.get_configspace(),
              run_id = run_id, nameserver='127.0.0.1',
              min_budget=250, max_budget=3000
           )
res = bohb.run()


17:06:11 wait_for_workers trying to get the condition
17:06:11 DISPATCHER: started the 'discover_worker' thread
17:06:11 DISPATCHER: started the 'job_runner' thread
17:06:11 DISPATCHER: Pyro daemon running on localhost:49906
17:06:11 DISPATCHER: Starting worker discovery
17:06:11 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
17:06:11 DISPATCHER: discovered new worker, hpbandster.run_plate_Gridsearch_1.worker.DESKTOP-NKTB8UL.2608.19764
17:06:11 HBMASTER: number of workers changed to 1
17:06:11 Enough workers to start this run!
17:06:11 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
17:06:11 adjust_queue_size: lock accquired
17:06:11 HBMASTER: starting run at 1648310771.895985
17:06:11 HBMASTER: adjusted queue size to (0, 1)
17:06:11 DISPATCHER: Finished worker discovery
17:06:11 start sampling a new configuration.
17:06:11 DISPATCHER: Trying to submit another job.
17:06:11 done sampling a new configuration.
17:06:11 HBMASTER: schedule new run fo


Total samples: 4320 
Batch size: 4320 
Total batches: 1 



17:07:05 WORKER: done with job (0, 0, 0), trying to register it.
17:07:05 WORKER: registered result for job (0, 0, 0) with dispatcher
17:07:05 DISPATCHER: job (0, 0, 0) finished
17:07:05 DISPATCHER: register_result: lock acquired
17:07:05 DISPATCHER: job (0, 0, 0) on hpbandster.run_plate_Gridsearch_1.worker.DESKTOP-NKTB8UL.2608.19764 finished
17:07:05 job_id: (0, 0, 0)
kwargs: {'config': {'activator': 'Addons>gelu', 'batch_size': 5000, 'denspt': 3, 'initial_lr': 0.0011225914254538842, 'loss': 'mae', 'numLayers': 2, 'numNeurons': 31, 'optimizer': 'SGD'}, 'budget': 333.3333333333333, 'working_directory': '.'}
result: {'loss': 171.14094707676958, 'info': {'L1': 171.14094707676958, 'L2': 269.3640637903978, 'MAX': 3.1972122192382812, 'TrainTime': 40.3125}}
exception: None

17:07:05 job_callback for (0, 0, 0) started
17:07:05 DISPATCHER: Trying to submit another job.
17:07:05 job_callback for (0, 0, 0) got condition
17:07:05 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiti


Total samples: 20000 
Batch size: 20000 
Total batches: 1 



17:07:11 DISPATCHER: Starting worker discovery
17:07:11 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
17:07:11 DISPATCHER: Finished worker discovery
17:08:11 DISPATCHER: Starting worker discovery
17:08:11 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
17:08:12 DISPATCHER: Finished worker discovery
17:09:12 DISPATCHER: Starting worker discovery
17:09:12 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
17:09:12 DISPATCHER: Finished worker discovery
17:10:12 DISPATCHER: Starting worker discovery
17:10:12 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
17:10:12 DISPATCHER: Finished worker discovery
17:11:12 DISPATCHER: Starting worker discovery
17:11:12 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
17:11:12 DISPATCHER: Finished worker discovery
17:12:12 DISPATCHER: Starting worker discovery
17:12:12 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
17:12:12 DISPATCHER: Finished worker discover

...

18:15:13 DISPATCHER: Starting worker discovery
18:15:13 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
18:15:13 DISPATCHER: Finished worker discovery



Epoch 02180: ReduceLROnPlateau reducing learning rate to 0.00045943728764541447.


18:16:13 DISPATCHER: Starting worker discovery
18:16:13 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
18:16:13 DISPATCHER: Finished worker discovery
18:17:13 DISPATCHER: Starting worker discovery
18:17:13 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
18:17:13 DISPATCHER: Finished worker discovery



Epoch 02670: ReduceLROnPlateau reducing learning rate to 0.00022971864382270724.


18:18:13 DISPATCHER: Starting worker discovery
18:18:13 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
18:18:13 DISPATCHER: Finished worker discovery
18:19:02 WORKER: done with job (0, 0, 2), trying to register it.
18:19:02 WORKER: registered result for job (0, 0, 2) with dispatcher
18:19:02 DISPATCHER: job (0, 0, 2) finished
18:19:02 DISPATCHER: register_result: lock acquired
18:19:02 DISPATCHER: job (0, 0, 2) on hpbandster.run_plate_Gridsearch_1.worker.DESKTOP-NKTB8UL.2608.19764 finished
18:19:02 job_id: (0, 0, 2)
kwargs: {'config': {'activator': 'Addons>softshrink', 'batch_size': 5000, 'denspt': 4, 'initial_lr': 0.007350996419417513, 'loss': 'mae', 'numLayers': 24, 'numNeurons': 91, 'optimizer': 'Nadam'}, 'budget': 3000.0, 'working_directory': '.'}
result: {'loss': 109.08237774613525, 'info': {'L1': 109.08237774613525, 'L2': 164.9391542913586, 'MAX': 4.751703577362891, 'TrainTime': 1190.703125}}
exception: None

18:19:02 job_callback for (0, 0, 2) started
18:19:02 D

In [7]:

# np.save('HpBandster_Results',res,allow_pickle=True)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

18:19:02 HBMASTER: shutdown initiated, shutdown_workers = True
18:19:02 WORKER: shutting down now!
18:19:02 DISPATCHER: Dispatcher shutting down
18:19:02 DISPATCHER: Trying to submit another job.
18:19:02 DISPATCHER: job_runner shutting down
18:19:02 DISPATCHER: discover_workers shutting down
18:19:02 DISPATCHER: 'discover_worker' thread exited
18:19:02 DISPATCHER: 'job_runner' thread exited
18:19:02 DISPATCHER: shut down complete
